In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.font_manager import FontProperties

# matplotlib.style.use('ggplot')
import matplotlib
# matplotlib.style.use('ggplot')
#matplotlib.style.use('default')
from numpy import *
import numpy as np
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (24, 18)
import subprocess,shlex,sys,select,re,StringIO
def readHive(sql):
    text = ''
    err = ''
    cmd = shlex.split('hive -e '+ '"set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;'+sql+'"')
    print >>sys.stderr, cmd
    sys.stderr.flush()
    proc = subprocess.Popen(cmd, bufsize=0, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    while proc.poll() is None:
        ins, outs, errs = select.select([proc.stdout, proc.stderr], [], [])
        if proc.stdout in ins:
            text += proc.stdout.read()
        if proc.stderr in ins:
            c = proc.stderr.read(1)
            if c == '\n':
                print >>sys.stderr, err
                sys.stderr.flush()
                err = ''
            else:
                err += c
    text += proc.stdout.read()
    print >>sys.stderr, proc.stderr.read()
    sys.stderr.flush()
    header, content = text.split('\n',1)
    header = re.sub('\w*\.','', header)
    return pd.read_table(StringIO.StringIO(header+'\n'+content), error_bad_lines = False)

import socket
import struct

def ip(x):
    lint=long(x)
    return socket.inet_ntoa(struct.pack("!I", lint))

import time
def ts(x):
    timeStamp = long(x)
    timeArray = time.localtime(timeStamp)
    return time.strftime("%Y-%m-%d %H:%M:%S", timeArray)



def f(x):
    x=str(x)
    tmp=x.split("//")
    if len(tmp)<=1:
        return x
    return '.'.join(x.strip().split('//')[1].split('/')[0].split('.')[-3:])
def spam_flag(x):
    if x==0:
        return "user"
    elif x==1:
        return "ip"
    elif x==2:
        return "domain"
    elif x==3:
        return "appid"
    elif x==5:
        return "request_id"
    else:
        return "normal"
def vendor_name(x):
    if x==1:
        return "Adx"
    elif x==5:
        return "Admob"
    elif x==2:
        return "Tanx"
    elif x==15:
        return "TanxMob"
    elif x==9:
        return "Bdx"
    elif x==21:
        return "BdxMob"
    elif x==24:
        return "GDTMob"
    elif x==31:
        return "GDT"
    else:
        return "other"
    
def preq(df):
    print "request_id-----------------"
    print df['request_id'].count()
    print df['request_id'].nunique()
def pzid(df):
    print "pid-----------------"
    print df['pid'].count()
    print df['pid'].nunique()
def pip(df):
    print "ip-----------------"
    print df['ip'].count()
    print df['ip'].nunique()
def pua(df):
    print "ua-----------------"
    print df['ua'].count()
    print df['ua'].nunique()

Populating the interactive namespace from numpy and matplotlib


In [3]:
c=readHive("select * from fraud_free.dclkv2 where year='2016' and month='12' and day='20' and vendor_id=1 and company_id='587'")

['hive', '-e', "set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;select * from fraud_free.dclkv2 where year='2016' and month='12' and day='20' and vendor_id=1 and company_id='587'"]

Logging initialized using configuration in file:/var/bh/lib/apache-hive-1.2.1-bin/conf/hive-log4j.properties
OK
 Fetched: 1927 row(s)



In [12]:
c['tip']=c['ip'].apply(ip)

In [22]:
c[c['ua'].str.contains('python-requests')]['tip'].value_counts()

106.75.29.15    570
106.75.29.17    521
106.75.29.16    506
106.75.29.14    218
Name: tip, dtype: int64

In [41]:
brod=readHive('select * from fraud_free.dclkv2 where year=2016 and month=12 and day=27 ')

['hive', '-e', 'set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;select * from fraud_free.dclkv2 where year=2016 and month=12 and day=27 ']

Logging initialized using configuration in file:/var/bh/lib/apache-hive-1.2.1-bin/conf/hive-log4j.properties
OK
conds, Fetched: 157211 row(s)



In [40]:
brod[brod['ua'].str.startswith('python-requests')].groupby(['hour','fraud_flag']).size()

hour  fraud_flag
13    5.0            81
14    5.0           152
15    5.0           156
dtype: int64

In [18]:
lspam=pd.read_table('/home/changbin/20161115spams',sep='\t',header=None,names=['vendor','spam','l'])
rspam=pd.read_table('/home/changbin/20161202spams',sep='\t',header=None,names=['vendor','spam','r'])

In [19]:
l=lspam.set_index(['vendor','spam'])
r=rspam.set_index(['vendor','spam'])

In [26]:
res=pd.concat([l,r],axis=[1,2])
res['diff']=res['r']-res['l']
res['ratio']=(res['r']/res['l']-1)*100

In [33]:
res

l           r        diff        ratio
vendor spam                                                 
1      2      3003853.0   4289347.0   1285494.0    42.794837
       8      2560725.0   2959768.0    399043.0    15.583204
       9      7275337.0   2454153.0  -4821184.0   -66.267501
       10      203828.0    207641.0      3813.0     1.870695
       13           NaN     65845.0         NaN          NaN
       14           NaN        23.0         NaN          NaN
       15      755709.0    907733.0    152024.0    20.116738
       16      782638.0    955563.0    172925.0    22.095145
       18          17.0         5.0       -12.0   -70.588235
       19      891891.0    671208.0   -220683.0   -24.743270
       20      891891.0    671208.0   -220683.0   -24.743270
       21       18364.0     18544.0       180.0     0.980179
       25        4405.0      7390.0      2985.0    67.763905
       26     1849834.0   1089517.0   -760317.0   -41.101904
       33    44625918.0  33699476.0 -10926442.0   -24.484520
       36      284748.0    337830.0     53082.0    18.641746
2      2     20787624.0  70922912.0  50135288.0   241.178540
       5          125.0         NaN         NaN          NaN
       8      3820215.0  13838368.0  10018153.0   262.240555
       9      4181916.0  11249499.0   7067583.0   169.003466
       10       86103.0   2111261.0   2025158.0  2352.017932
       14         188.0      6457.0      6269.0  3334.574468
       15     1263957.0   5583768.0   4319811.0   341.768826
       16     1364581.0   5656528.0   4291947.0   314.524898
       18         130.0      3609.0      3479.0  2676.153846
       19     6311201.0  24352989.0  18041788.0   285.869330
       20     2363941.0  10026330.0   7662389.0   324.136220
       21      226350.0   1577209.0   1350859.0   596.800972
       25      629669.0    909689.0    280020.0    44.470984
       26     5009141.0  13714235.0   8705094.0   173.784168
...                 ...         ...         ...          ...
24     12     3518215.0   1420890.0  -2097325.0   -59.613327
       14          45.0        18.0       -27.0   -60.000000
       15     7874702.0    243012.0  -7631690.0   -96.914017
       16     6439852.0    217807.0  -6222045.0   -96.617826
       18        2749.0         NaN         NaN          NaN
       19    19571181.0   1909337.0 -17661844.0   -90.244140
       20     6975191.0    322838.0  -6652353.0   -95.371625
       25      970287.0     30079.0   -940208.0   -96.899989
       26    17851165.0   7312196.0 -10538969.0   -59.037990
       28         305.0         NaN         NaN          NaN
       33    21367010.0   3152693.0 -18214317.0   -85.245044
31     5           43.0         NaN         NaN          NaN
       8       591337.0         NaN         NaN          NaN
       9       436624.0       500.0   -436124.0   -99.885485
       15     5077850.0        41.0  -5077809.0   -99.999193
       16     7092054.0        39.0  -7092015.0   -99.999450
       19     9918517.0     10044.0  -9908473.0   -99.898735
       20     2928862.0      4216.0  -2924646.0   -99.856053
       25      338427.0         3.0   -338424.0   -99.999114
       26    46198065.0      6045.0 -46192020.0   -99.986915
       33    18389670.0     32925.0 -18356745.0   -99.820959
38     2          843.0         NaN         NaN          NaN
57     8        12951.0         NaN         NaN          NaN
       9        20588.0         NaN         NaN          NaN
       15       88497.0         NaN         NaN          NaN
       16       88497.0         NaN         NaN          NaN
       19      222636.0         NaN         NaN          NaN
       20      197037.0         NaN         NaN          NaN
       26       51553.0         NaN         NaN          NaN
       33       89078.0         NaN         NaN          NaN

[191 rows x 4 columns]

In [32]:
r.query('vendor==75')

,,r
vendor,spam,


In [ ]:
cre=pd.read_table('/home/changbin/rtbmiss_criteo_click',sep='\t',names=['zid', 'ts', 'hour', 'minute', 'url', 'refer', 'ip', 'ua'])

In [ ]:
cre[['zid','ip','ua']].to_csv('/home/changbin/zid',sep='\t',index=None)

In [ ]:
cre['ip_']=cre['ip'].apply(ip)
cre['ts_']=cre['ts'].apply(ts)

In [ ]:
cre[['ip_','ua']].to_csv('/home/changbin/ip_ua',sep='\t')

In [ ]:
res=readHive("select * from archive.rtbv2 a join changbin.zid_ip_ua b on a.ip=b.ip and a.user_agent=b.ua where a.year=2016 and a.month=11 and a.day=28")

In [ ]:
miss=readHive("select * from changbin.zid_ip_ua")

In [ ]:
resall=readHive("select * from archive.rtbv2 a join changbin.zid_ip_ua_all b on a.ip=b.ip and a.user_agent=b.ua where a.year=2016 and a.month=11 and a.day=28")

In [ ]:
resadgroup=readHive("select distinct request_id, zid,`timestamp` from archive.rtbv2 lateral view explode(ad_response[0].bid_code) ids as id1 where year=2016 and month=11 and day=28 and hour>=17 and hour<19 and in_file(zid,'/home/changbin/criteo_all_click_zids') and id1.adgroup_id=26778 and vendor_id in (9);")

In [ ]:
resseg=readHive("select distinct request_id, zid,`timestamp` from archive.rtbv2 lateral view explode(ad_response[0].matched_segments) ids as id1 where year=2016 and month=11 and day=28 and in_file(zid,'/home/changbin/criteo_all_click_zids') and id1.id=4889 and vendor_id in (9);")

In [ ]:
# a.zid not in ('ce4d970c5c15abb5b4950de43c514fcf','e09947276b54a2304835ae8a65da8b10')
reszid=readHive("select * from archive.rtbv2 a join changbin.zid_ip_ua_all b on a.zid=b.zid where a.zid=a.zid and a.year=2016 and a.month=11 and a.day=28 and a.zid not in ('ce4d970c5c15abb5b4950de43c514fcf','e09947276b54a2304835ae8a65da8b10')")

In [ ]:

resbid=pd.read_table('/home/changbin/rtb_day_winprice_log',header=None,names=['request_id', 'zid', 'timestamp', 'hour', 'min', 'compaign', 'bid_price', 'win_price', 'is_displayed', 'is_click'])
reswin=resbid.query('win_price>0')
reswin

In [ ]:
#proc datetime
import datetime
def to_date(x):
    ts=int(x)+3600*8
    return pd.to_datetime(ts,unit='s',box=True)
resbid['bid']=resbid['timestamp'].apply(to_date)
reswin['win']=reswin['timestamp'].apply(to_date)
resseg['seg']=resseg['timestamp'].apply(to_date)
resadgroup['adgroup']=resadgroup['timestamp'].apply(to_date)
reszid['brodcast_bin']=reszid['timestamp'].apply(to_date)#pd.to_datetime(reszid['timestamp'],unit='s',box=True)
reszid['criteo_bin']=reszid['ts'].apply(to_date)#pd.to_datetime(reszid['ts'],unit='s',box=True)
reszid.rename(columns={'zid.1': 'zid_y','hour.1': 'hour_y','refer.1': 'refer_y','ip.1': 'ip_y'}, inplace=True)
# reszid.query('zid=="2d84624b3a45dbf597fe76c3ad39878f"')

In [ ]:
temp.query(target).ip
print ip(3071416885)

In [ ]:
allwinzids =['2d84624b3a45dbf597fe76c3ad39878f',
'4647895d6ce90b478e135eea25fa9b70',
'6ffe01eea158ccd2bb7cd2a24a61cdee',
'71f1afdd04c4dddd6d7bd6dae90ad2e4',
'78602be560ac24c392e1459afe01af61',
'7e3468dcae25de01045f6cec73759d45',
 'a572bfdeff58fc099a48c4bd5c1ab44d',
 'ab95641ba92bc6f24738c2fb77cce6c5',
 'c83e48111462b8490cbf183e92db26dd',
 'cdce06ff3fa38a712b3e61918c9cf9a1',
 'd27bec7275f17d297087ce436c7f9d88',
 'dc98ccd4b70ad3a0856bb87655300292',
 'eb722a4bb298575c99223c86cfcc1e8b',
 'ebc3046dbf22efe6afab88b4efba6a97']
notwinzids=['36312029f7c8c42d1e476b9951d92550',
'e10e69e7ed64e77ee1755e241f1e4bd6',
'931158a73f08ab6ffb2de679347cf039',
'c4b69b3794266f1724035c36436fcd56',
'58b370df10d70d1a662f1c5713911c32',
'fcb3ecbeb5406668541c397e34036836',
'da1abf792cfe0be71db4930be29034a3',
'35168e0300e022f746dd3aefeadb779d'
]
freqzids=['0180f0075bfd64bf474ff0733a2433f1',
'12887720558e379c15d9d2bbca4a8e04',
'1ec07e5fcb66196004a1c25cb297651f',
'29c30b07469f0c95b5e5f4dc7aae88d0',
'2e402d28d0532f0fbac1c13963d202a7',
'2f8dacbb46074b5a0f150cfe16d234fe',
'32aa12b549541e38b3c0ab05578db0a8',
'367560d5bb2fb08b49a28ab3ca858eed',
'39f7b1b7460753d153db95bbb9d50396',
'3e609bdfeda47596a5f326a0083b8fc3',
'3f9a24c98f37147b049bdd046db5cc8e',
'45dac31354776ab7c120551024620e5c',
'46ab8ad97118d120fb1a455edb77766a',
'496f59d3debb1b961e736263b0c2a6c5',
'4f7e34552939534bf628313dc3db7ca3',
'53b3ca586a88cade594bbea5e6cded1f',
'56aa7cf88418e5649023fbe693b81ee9',
'5d1dd575ed2d551604c8df4b5a418391',
'646778f7758b7c825b08f4013156180f',
'6bda6432f6f34338b0cf80b44df0d4d4',
'74901db70cbeda00a8b731d20cd9ec6c',
'7a95f2ad07134da6313d7acdafe52ef8',
'890994e503c7603070b61053ca1992fa',
'892993c434dd202ba0d3119919f8c3a8',
'8a6d6b983c3a43fe25692c4419475d54',
'9e7e76c17f0ae5ac60fdb6dce56413ed',
'a334afbdccf01f7d1b3f23fe7d95dd91',
'a5b134dbd06eb0399b40570a4028139c',
'ad2ef2f8c72eb3b3add92d87c7031b49',
'b87805ada2508de6758bbe835bb240b4',
'c2ea7bff3565603d583263e49f5ea54f',
'ca030776ca87ddf232ea091aa3db0df7',
'ce4d970c5c15abb5b4950de43c514fcf',
'd5746c6488ea8d42a516698be0014a7d',
'e09947276b54a2304835ae8a65da8b10',
'e18cdc77bd70650d7113f279b52b370c',
'e97d4605b57faf445cb3d57ea86f7dea',
'f0b65b4844613a80c470383beb67f6ad',
'ff209c1e263bd72e92725e52b4407d74']

In [ ]:
from pylab import rcParams
import matplotlib
%matplotlib inline   
%pylab inline
matplotlib.style.use('ggplot')
window='300s'
zids_set=[allwinzids,notwinzids,freqzids]
tags=['allwin','notallwin','freq']
for i in range(len(zids_set)):
    zids=zids_set[i]
    tag=tags[i]
    for zid in zids:
        # for zid in zids:
        target='zid=="'+zid+'"'
        #selection
        print "zid:",zid
        segtemp=resseg.query(target)
        adgrouptemp=resadgroup.query(target)
        bidtemp=resbid.query(target)
        wintemp=reswin.query(target)
        temp=reszid.query('zid==zid_y & vendor_id==9 &'+target)
        #resample
        seged=segtemp.groupby(['seg'])['request_id'].nunique().resample(window).sum().reset_index().set_index('seg')
        adgrouped=adgrouptemp.groupby(['adgroup'])['request_id'].nunique().resample(window).sum().reset_index().set_index('adgroup')
        bid=bidtemp.groupby(['bid'])['request_id'].nunique().resample(window).sum().reset_index().set_index('bid')
        win=wintemp.groupby(['win'])['request_id'].nunique().resample(window).sum().reset_index().set_index('win')
        l=temp.groupby(['brodcast_bin'])['request_id'].nunique().resample(window).sum().reset_index().set_index('brodcast_bin')#.tz_localize("Asia/Shanghai")
        r=temp.groupby(['criteo_bin'])['request_id'].nunique().resample(window).sum().reset_index().set_index('criteo_bin')#.tz_localize("Asia/Shanghai")
        r['request_id']=l['request_id'].mean()
        seged.rename(columns={'request_id':'seg'}, inplace=True)
        adgrouped.rename(columns={'request_id':'adgroup'}, inplace=True)
        bid.rename(columns={'request_id':'bid'}, inplace=True)
        win.rename(columns={'request_id':'win'}, inplace=True)
        l.rename(columns={'request_id':'cap'}, inplace=True)
        r.rename(columns={'request_id':'criteo'}, inplace=True)
        phase=['2016-11-28 16:30:00','2016-11-28 20:00:00']
        res=pd.concat([l,seged,adgrouped,bid,win,r],axis=1)
        rcParams['figure.figsize'] = 24, 10
        fig = plt.figure()
        ax=res.sort_index()[phase[0]:phase[1]].plot(kind='bar',title=tag+'-'+zid)
        ax.get_figure().savefig('/home/changbin/timedist/'+tag+'-'+zid+'.png')
#         plt.close(fig)
        # res.sort_index().plot.bar()

In [ ]:
# 36312029f7c8c42d1e476b9951d92550
# e10e69e7ed64e77ee1755e241f1e4bd6
# 931158a73f08ab6ffb2de679347cf039
# c4b69b3794266f1724035c36436fcd56
# 58b370df10d70d1a662f1c5713911c32
# fcb3ecbeb5406668541c397e34036836
# da1abf792cfe0be71db4930be29034a3
# 35168e0300e022f746dd3aefeadb779d
window='600s'
target='zid=="da1abf792cfe0be71db4930be29034a3"'
#selection
print "zid:",zid
segtemp=resseg.query(target)
adgrouptemp=resadgroup.query(target)
bidtemp=resbid.query(target)
wintemp=reswin.query(target)
temp=reszid.query('zid==zid_y & vendor_id==9 &'+target)
#resample
seged=segtemp.groupby(['seg'])['request_id'].nunique().resample(window).sum().reset_index().set_index('seg')
adgrouped=adgrouptemp.groupby(['adgroup'])['request_id'].nunique().resample(window).sum().reset_index().set_index('adgroup')
bid=bidtemp.groupby(['bid'])['request_id'].nunique().resample(window).sum().reset_index().set_index('bid')
win=wintemp.groupby(['win'])['request_id'].nunique().resample(window).sum().reset_index().set_index('win')
l=temp.groupby(['brodcast_bin'])['request_id'].nunique().resample(window).sum().reset_index().set_index('brodcast_bin')#.tz_localize("Asia/Shanghai")
r=temp.groupby(['criteo_bin'])['request_id'].nunique().resample(window).sum().reset_index().set_index('criteo_bin')#.tz_localize("Asia/Shanghai")
r['request_id']=l['request_id'].mean()
seged.rename(columns={'request_id':'seg'}, inplace=True)
adgrouped.rename(columns={'request_id':'adgroup'}, inplace=True)
bid.rename(columns={'request_id':'bid'}, inplace=True)
win.rename(columns={'request_id':'win'}, inplace=True)
l.rename(columns={'request_id':'cap'}, inplace=True)
r.rename(columns={'request_id':'criteo'}, inplace=True)
phase=['2016-11-28 17:00:00','2016-11-28 20:00:00']
res=pd.concat([l,seged,adgrouped,bid,win,r],axis=1)
rcParams['figure.figsize'] = 24, 10
fig = plt.figure()
res.sort_index()[phase[0]:phase[1]].plot(kind='bar',title=zid)
fig.savefig('/home/changbin/timedist/'+zid+'.png')

In [ ]:
# date_index=pd.date_range('11/28/2016', periods=144, freq='1800s')






# l['r']=r['request_id']
# l.rename(columns={'request_id':'capture','r':'criteo'}, inplace=True)
# l.plot(kind='bar')


In [ ]:
temp=resall.query('zid==zid_y & vendor_id==9 & zid=="4990d4acff9055cc68b6801ee53bb608"')
temp.groupby(['brodcast_bin']).sum()['vendor_id'].plot()
temp.groupby(['criteo_bin']).sum()['vendor_id'].plot()


In [ ]:
r=pd.date_range('11/28/2016', periods=144, freq='600s')
r

In [ ]:
#res.query('hour==18 & vendor_id==9 & zid=="b3dff87c0cc04a163111d8d345e0a4c2"')['ad_response'].iloc[0]
j=pd.merge(res,miss,how='outer',left_on='zid.1',right_on='zid')
j.query('request_id!=request_id')['zid_y'].nunique()



In [ ]:
j.query('request_id==request_id & zid_x==zid_y')['zid_x'].nunique()

In [ ]:
res.rename(columns={'zid.1': 'zid_y'}, inplace=True)

In [ ]:
res.query('zid==zid_y &vendor_id==9')['zid_y'].value_counts()

In [ ]:
# print 'miss',j.query('request_id==request_id')['zid_y'].nunique()
# print miss['zid'].nunique()
res.query('vendor_id==9 & zid==zid & zid==zid_y ')['hour'].value_counts()

In [ ]:
s21=readHive("select vendor_id, spams, count(1) from archive.rtbv2 lateral view explode(ad_response[0].spam_ids) spam as spams where year=2016 and month=11 and day=21 group by vendor_id ,spams;")
s27=readHive("select vendor_id, spams, count(1) from archive.rtbv2 lateral view explode(ad_response[0].spam_ids) spam as spams where year=2016 and month=11 and day=27 group by vendor_id ,spams;")


In [ ]:
#m=pd.merge(s21,s27,on=['vendor_id','spams'],how='inner')
r21=readHive("select vendor_id, size(ad_response[0].spam_ids)>0, count(1) from archive.rtbv2 where year=2016 and month=11 and day=21 group by vendor_id,size(ad_response[0].spam_ids)>0;")
r27=readHive("select vendor_id, size(ad_response[0].spam_ids)>0, count(1) from archive.rtbv2 where year=2016 and month=11 and day=27 group by vendor_id,size(ad_response[0].spam_ids)>0;")


In [ ]:
m['r']=(m['_c2_y']-m['_c2_x'])/m['_c2_x']

In [ ]:
m.query('vendor_id==2')

In [ ]:
yhd=readHive("select * from fraud_free.dclkv2 where year=2016 and month=11 and day=21 and adgroup_id=26731")

In [ ]:
cre=readHive("select * from fraud_free.dclkv2 where year=2016 and month=11 and day=28 and zid='1c4766ddb3f2bb5cd064295efa2b15fc'")

In [ ]:
cre

In [ ]:
yhd['domain']=yhd['url'].apply(f)
preq(yhd.query('fraud_flag==5'))

In [ ]:

yhd.query('fraud_flag==5')['domain'].value_counts()

In [ ]:
imphour=readHive("select * from fraud_free.dimpv2 where year=2016 and month=11 and day=11 and hour>=19 and hour<=20")
clkhour=readHive("select * from fraud_free.dclkv2 where year=2016 and month=11 and day=11 and hour>=19 and hour<=20")
clk=readHive("select * from fraud_free.dclkv2 where year=2016 and month=11 and day=11")

In [ ]:
clk['vendor_name']=clk['vendor_id'].apply(vendor_name)
clk['domain']=clk['url'].apply(f)
clk['wash_label']=clk['fraud_flag'].apply(spam_flag)

clkhour['vendor_name']=clkhour['vendor_id'].apply(vendor_name)
clkhour['domain']=clkhour['url'].apply(f)
clkhour['wash_label']=clkhour['fraud_flag'].apply(spam_flag)

In [ ]:
# print clk.replace(np.nan, 'normal').groupby(['domain','fraud_flag']).size().order(ascending=False)
print clk.groupby(['vendor_name','wash_label']).size().order(ascending=False).head(100)
print clk.groupby(['domain','wash_label']).size().order(ascending=False).head(100)
print clk.groupby(['company_id','wash_label']).size().order(ascending=False).head(100)
preq(clk.query('fraud_flag!=fraud_flag'))
pzid(clk.query('fraud_flag!=fraud_flag'))
pip(clk.query('fraud_flag!=fraud_flag'))

preq(clk.query('fraud_flag==fraud_flag'))
pzid(clk.query('fraud_flag==fraud_flag'))
pip(clk.query('fraud_flag==fraud_flag'))
# clk['domain'].value_counts()
# clk.groupby['vendor_id','fraud_flag'].size()

In [ ]:
domain=clk.query('fraud_flag!=fraud_flag').groupby(['domain','wash_label']).size().reset_index()
domain['count']=domain[0]
count_bins=[0 ,1000,2000 ,4000, 8000 ,16000,  32000, 64000,100000]
print count_bins
domain['bins']=pd.cut(domain['count'],count_bins)
domain.groupby(["bins"])['count'].size()
print domain['count'].sum()

In [ ]:
clk.groupby('wash_label').size()

In [ ]:
preq(clk.query('domain=="tv.2345.com"'))
preq(clk.query('domain=="tv.2345.com" & fraud_flag==fraud_flag'))
preq(clk.query('domain=="www.ku6.com"'))
preq(clk.query('domain=="www.ku6.com" & fraud_flag==fraud_flag'))
clk.query('fraud_flag==fraud_flag')['fraud_flag'].value_counts()

In [ ]:
app=pd.read_table("../app_data",header=0)#None,names=['day','vendor','err','count'])
# clk_bins=np.arange(0,1,0.001)

In [ ]:
app.query('allimp>50000000')

In [ ]:
err.query("err ==0").groupby(['day','vendor'])['count'].sum()

In [ ]:
temp=clkhour.query('fraud_flag!=fraud_flag')
preq(temp)
pzid(temp)
pip(temp)
pua(temp)

In [ ]:
temp['vendor_name']=temp['vendor_id'].apply(vendor_name)
temp['domain']=temp['url'].apply(f)
temp['wash_label']=temp['fraud_flag'].apply(spam_flag)
print temp.groupby(['vendor_name','wash_label']).size().order(ascending=False).head(100)
print temp.groupby(['domain','wash_label']).size().order(ascending=False).head(100)
print temp.groupby(['company_id','wash_label']).size().order(ascending=False).head(100)

In [ ]:
print clk.query('fraud_flag==fraud_flag & vendor_id==9')['domain'].value_counts()
# 21      16671376369665264848
# 2       mm_108636149_8862290_42786782
# 2       mm_26632206_2690592_13290789
# 2       mm_26632206_2690592_14738448
# 9       9223372032562359521
# 9       9223372032562452846
# 9       9223372032562467558

# 9223372032561520956    2034
# 9223372032562020292    1811
# 9223372032562082811    1784
# 9223372032562004323    1601
# 9223372032562530560    1510
# 9223372032562359521    1463
# 9223372032562071233    1403
# 9223372032562452846    1388
# 9223372032562416712    1317
# 9223372032562531153    1308
# 9223372032560087934    1228
# 9223372032560501554    1052
# 9223372032561548281    1030

In [ ]:
ctr_bins=np.arange(0,1,0.001)
cpc_bins=np.arange(0,100,0.05)

In [ ]:
c=pd.read_table("/home/changbin/blobtest/appid_ctr_cpc_imp",header=0)
c.query('allimp>1000')['cpc'].value_counts(bins=cpc_bins)
# c.query('allimp>1000')['ctr'].value_counts(bins=ctr_bins)
c['ctr_bins']=pd.cut(c['ctr'],ctr_bins)
c['cpc_bins']=pd.cut(c['cpc'],cpc_bins)

In [ ]:
ku6=readHive("select * from fraud_free.dclkv2 where year=2016 and month=11 and day=10 and url like '%ku6.com%'")

In [ ]:
ku6.replace(np.nan,-1).groupby('fraud_flag').size()
pzid(ku6.query('fraud_flag==fraud_flag'))